# Selective flow constraints with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

The [Operations Problems example]](../../notebook/PowerSimulations_examples/sequential_simulations.ipynb)
shows the basic building blocks of building optimization problems with PowerSimulations.jl.
This example shows how to customize the enforcement of branch flow constraints as is common
when trying to build large scale simulations.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations

### Optimization packages
For this simple example, we can use the Cbc solver with a relatively relaxed tolerance.

In [3]:
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Create a `System` from RTS-GMLC data
We can just use the
[parsing tabular data example.](../../notebook/PowerSystems_examples/parse_tabulardata.jl)

In [4]:
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "PowerSystems_examples", "parse_tabulardata.jl"))

┌ Info: Parsing csv files in Reserves ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/0ktWM/src/parsers/power_system_table_data.jl:130
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/0ktWM/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/0ktWM/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/0ktWM/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/0ktWM/src/parsers/power_system_table_data.jl:134
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/0ktWM/src/parsers/pow

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,3
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,73
3,GenericBattery,Storage <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
4,HVDCLine,DCBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,1
5,HydroDispatch,HydroGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,20
6,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,104
7,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,21
8,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,51
9,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any,30


### Selecting flow limited lines
Since PowerSimulations will apply constraints by component type (e.g. Line), we need to
change the component type of the lines on which we want to enforce flow limits. So, let's
change the device type of certain branches from Line to MonitoredLine differentiate
treatment when we build the model. Here, we can select inter-regional lines, or lines
above a voltage threshold.

In [5]:
for line in get_components(Line, sys)
    if (get_basevoltage(get_from(get_arc(line))) >= 230.0) &&
       (get_basevoltage(get_to(get_arc(line))) >= 230.0)
        #if get_area(get_from(get_arc(line))) != get_area(get_to(get_arc(line)))
        @info "Changing $(get_name(line)) to MonitoredLine"
        convert_component!(MonitoredLine, line, sys)
    end
end

┌ Info: Changing C31-2 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B26 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing A32-2 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing CA-1 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B34 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B19 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B31-2 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing C30 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B27 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing A21 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing A32-1 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing B29 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing A18 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing C24 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing C28 to MonitoredLine
└ @ Main.##426 string:5
┌ Info: Changing A29 to MonitoredLine
└ @ Main.##426 string:5

## Build an `OperationsProblem`

In [6]:
uc_prob =
    UnitCommitmentProblem(sys, optimizer = solver, horizon = 24, slack_variables = true)

┌ Info: PowerSimulations.FixedOutput for PowerSystems.HydroDispatch is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Info: PowerSimulations.FixedOutput for PowerSystems.RenewableFix is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Info: PowerSimulations.FixedOutput for PowerSystems.HydroDispatch is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Info: PowerSimulations.FixedOutput for PowerSystems.RenewableFix is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Warning: The data do


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      ILoads:
        device_type = PowerSystems.InterruptibleLoad
        formulation = PowerSimulations.InterruptiblePowerLoad
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.FixedOutput
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalBasicUnitCommitment
      DistRE:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.FixedOutput
      Hydro:
        device_type = PowerSystems.HydroEnergyReservoir
        formulation = PowerSimulations.HydroDispatchReservoirFlow
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RE:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
  branches: 
      T:
        de

The above function defaults to a basic `CopperPlatePowerModel`, ror now, let's just
choose a standard DCOPF (B-theta) formulation.

In [7]:
set_transmission_model!(uc_prob, DCPPowerModel) #TODO: rm this and add network = DCPPowerModel to above when PSIMA-138 is tagged

┌ Info: PowerSimulations.FixedOutput for PowerSystems.HydroDispatch is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Info: PowerSimulations.FixedOutput for PowerSystems.RenewableFix is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Info: PowerSimulations.FixedOutput for PowerSystems.HydroDispatch is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Info: PowerSimulations.FixedOutput for PowerSystems.RenewableFix is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Warning: The data do

Let's change the formulation of the `Line` components to an unbounded flow formulation.
This formulation still enforces Kirchoff's laws, but does not apply flow constraints.

In [8]:
set_branch_model!(uc_prob, :L, DeviceModel(Line, StaticLineUnbounded))

┌ Info: PowerSimulations.FixedOutput for PowerSystems.HydroDispatch is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Info: PowerSimulations.FixedOutput for PowerSystems.RenewableFix is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Info: PowerSimulations.FixedOutput for PowerSystems.HydroDispatch is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Info: PowerSimulations.FixedOutput for PowerSystems.RenewableFix is not compatible with the provision of reserve services
└ @ PowerSimulations /Users/cbarrows/Documents/repos/PowerSimulations.jl/src/services_models/services_constructor.jl:39
┌ Warning: The data do

Notice that there is no entry for `MonitoredLine` branches. So, let's add one.

In [9]:
construct_device!(uc_prob, :ML, DeviceModel(MonitoredLine, StaticLine))

Solve the relaxed problem

In [10]:
solve!(uc_prob)

,Time,B8,B13-2,C11,C8,C9,B1
,Dates…,Float64,Float64,Float64,Float64,Float64,Float64
1,2020-01-01T00:00:00,-0.277532,-0.5046,-0.548086,-0.507371,-0.00250536,-0.111409
2,2020-01-01T01:00:00,-0.270273,-0.497116,-0.522975,-0.515989,-0.192952,-0.117793
3,2020-01-01T02:00:00,-0.268661,-0.494984,-0.51131,-0.473767,0.0279218,-0.1167
4,2020-01-01T03:00:00,-0.276273,-0.502087,-0.504389,-0.473559,0.0111105,-0.142633
5,2020-01-01T04:00:00,-0.284333,-0.510348,-0.510519,-0.481799,-0.0216968,-0.137433
6,2020-01-01T05:00:00,-0.313719,-0.540292,-0.539189,-0.472768,-0.0261576,-0.146085
7,2020-01-01T06:00:00,-0.181135,-0.585654,-0.600087,-0.520889,-0.136589,-0.447164
8,2020-01-01T07:00:00,-0.308286,-0.563104,-0.64062,-0.585036,-0.390839,-0.067066
9,2020-01-01T08:00:00,-0.300084,-0.553639,-0.686887,-0.659103,-0.566702,-0.0642643


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*